In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

In [4]:
myTable = soup.find('table',{'class':'wikitable sortable'})
print (myTable.prettify()[0:250])

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </t


In [5]:
titles = []
for title in myTable.findAll('th'):
    titles.append(title.text)
print(titles)

['Postcode', 'Borough', 'Neighbourhood\n']


In [8]:
rowlen = len(myTable.findAll('tr'))
#np.arange(0,rowlen+1,1)

In [9]:
rowlen = len(myTable.findAll('tr'))
df = pd.DataFrame(index=np.arange(0,rowlen+1,1),columns=list(titles))
row_marker = 0
for row in myTable.findAll('tr'): #rows in tr
    column_marker = 0
    columns = row.findAll('td') # columns in td
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text() #iat returns value from row and col position
        column_marker +=1
    row_marker +=1

df.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


There are no Boroughs with Neighbourhoods containing "Not assigned value".

In [10]:
df.loc[(df["Borough"] != 'Not assigned') & (df["Neighbourhood\n"] == 'Not assigned')]

,Postcode,Borough,Neighbourhood


In [11]:
df.dropna(axis = 0,inplace=True)
# drop rows with NaN values

drop postcodes wich have to Borough and Neighbourhood assigned

In [12]:
df = df[df.Borough != 'Not assigned']

In [13]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n


Remove "\n" from end of string.

In [14]:
df = df.replace('\n','',regex=True)

In [15]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [16]:
df.describe()

,Postcode,Borough,Neighbourhood
count,212,212,212
unique,103,11,210
top,M8Y,Etobicoke,Runnymede
freq,8,45,2


In [17]:
df.loc[df["Postcode"] == 'M5A']

,Postcode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [18]:
print(df.columns)

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')


The neighbourhood column name needs to be changed as it contains "\n"

In [19]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace = True)

Dataframe is grouped by Postcode and Borough by joining the Neighbourhood strings by comma

In [20]:
df2 = df.groupby(["Postcode","Borough"])["Neighbourhood"].apply(', '.join).reset_index()

In [21]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
df.shape

(212, 3)